In [304]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
import random
import math
import operator

# Handle Data

In [305]:
with open('iris.data.txt', 'r') as csvfile:
    lines = csv.reader(csvfile)
    row = next(lines)
    for row in lines :
        print (', '.join(row))
        

4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4, 1.5, 0.4, Iris-setosa
5.2, 4.1

In [306]:

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
                
            else:
                testSet.append(dataset[x])
        #Converting categorical to numerical so we can use knn
            switcher ={   
            'Iris-setosa':0,
             'Iris-versicolor':1,
            'Iris-virginica':2    
       

    }
            dataset[x][4]=switcher.get(dataset[x][4],"Invalid ") 
                 

In [307]:
trainingSet=[]

testSet=[]

loadDataset('iris.data.txt', 0.66, trainingSet, testSet)

print ('Train: ' + repr(len(trainingSet)))

print ('Test: ' + repr(len(testSet)) )


Train: 102
Test: 48


# Similarity

In [308]:
def euclideanDistance(instance1, instance2, length): 
    tot=0
    for i in range(length):
        tot=tot+math.pow(instance1[i]-instance2[i],2)
    return math.sqrt(tot)

# Neighbors


In [309]:
def getNeighborsEuc(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

# Response

In [310]:
# From the given 5 neighbors we will let the value to vote fot the most appropriate one
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1] 
        if response in classVotes:
             classVotes[response]+=1
            
        else: 
            classVotes[response]=1
            
    
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# Accuracy

In [311]:
def getAccuracy(testSet, predictions):
    correct=0
   
    for i in range(len(predictions)):
        
        if(predictions[i]==testSet[i][-1]):
            correct+=1
          
    return (correct/float(len(testSet))) * 100.0


# Euclidean distance

In [312]:
def KNN_main_eucl (file,p,k): 
    trainingSet=[]
    testSet=[]
    response=[]

    loadDataset(file, p, trainingSet, testSet)
    print ('Train: ' + repr(len(trainingSet)))

    print ('Test: ' + repr(len(testSet)) )
    for i in range(len(testSet)):
        
        neighbors =getNeighborsEuc(trainingSet,testSet[i], k)
        response.append(getResponse(neighbors))    
       
    acc=getAccuracy(testSet,response)
    return acc
    

In [313]:
acc=KNN_main_eucl  ('iris.data.txt',0.66,5)
print("accuracy= ",acc)

Train: 98
Test: 52
accuracy=  92.3076923076923


# Manhattan distance 

In [314]:
def manhattanDistance(instance1, instance2, length): 
    tot=0
    for i in range(length):
        tot=tot+abs(instance1[i]-instance2[i])
    return tot

In [315]:
def getNeighborsManhattan(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = manhattanDistance(testInstance, trainingSet[x],4)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [316]:
def KNN_main_Manhattan (file,p,k): 
    trainingSet=[]
    testSet=[]
    response=[]

    loadDataset(file, p, trainingSet, testSet)
    print ('Train: ' + repr(len(trainingSet)))

    print ('Test: ' + repr(len(testSet)) )
    for i in range(len(testSet)):
        
        neighbors =getNeighborsManhattan(trainingSet,testSet[i], k)
        response.append(getResponse(neighbors))    
       
    acc=getAccuracy(testSet,response)
    return acc

In [317]:
acc= KNN_main_Manhattan  ('iris.data.txt',0.66,5)
print("accuracy= ",acc)

Train: 100
Test: 50
accuracy=  94.0


# Minkowski distance

In [318]:

from math import *
from decimal import Decimal 


def minkowski_distance(x, y, p_value): 
    res=0
    for i in range (p_value):
        res+=(abs(x[i]-y[i]))**4
    return res**(1/4) 
      
 

In [319]:
def getNeighbors_minkowski(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = minkowski_distance(testInstance, trainingSet[x],4)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [320]:
def KNN_main_Minkowski (file,p,k): 
    trainingSet=[]
    testSet=[]
    response=[]

    loadDataset(file, p, trainingSet, testSet)
    print ('Train: ' + repr(len(trainingSet)))

    print ('Test: ' + repr(len(testSet)) )
    for i in range(len(testSet)):
        
        neighbors =getNeighbors_minkowski(trainingSet,testSet[i], k)
        response.append(getResponse(neighbors))    
       
    acc=getAccuracy(testSet,response)
    return acc

In [321]:
acc=KNN_main_Minkowski  ('iris.data.txt',0.66,5)
print("accuracy= ",acc)

Train: 98
Test: 52
accuracy=  100.0
